In [6]:
import gradio as gr
import requests
import re

# Azure OpenAI 설정
AZURE_OPENAI_ENDPOINT = 'https://1507-ai.openai.azure.com/'
AZURE_OPENAI_API_KEY = '22646ec32d23457ca3ed253eefc5342f'
DEPLOYMENT_NAME = '1507-gpt4o'

# Azure OpenAI API 호출 함수  
def chatgpt_response(prompt, history):  
    print(prompt, history)
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    messages = []

    # System
    messages.append({
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "너는 서울 구로구의 쉼터에 대한 전문가야. 유저가 작성한 장소에서 어느 곳이 가까운지, 또 쉼터에 대한 정보를 알려줘"
            }
        ]
    })

    # Assistant
    if len(history) > 0:
        for text in history[0]:
            messages.append({
                "role": "assistant",
                "content": [
                    {
                        "type": "text",
                        "text": text
                    }
                ]
            })

    # User
    messages.append({
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": prompt
            }
        ]
    })

    payload = {
        "messages": messages,
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 4096
    }

    response = requests.post(  
        f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",  
        headers=headers,  
        json=payload  
    )  
    
    result = response.json()  
    bot_response = result['choices'][0]['message']['content'].strip()
    history.append((prompt, bot_response))  
    return bot_response, history

# 텍스트에서 한국어, 영어, 숫자만 남기는 함수
def filter_text(text):
    filtered_text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)
    return filtered_text

# Azure TTS 설정 및 호출 함수
def get_token():
    endpoint = 'https://eastus.api.cognitive.microsoft.com/sts/v1.0/issueToken'
    api_key = '5fb3d580ca744ae7a95b9fbe10307ec3'

    headers = {
        "Ocp-Apim-Subscription-Key": api_key
    }
    
    response = requests.post(endpoint, headers=headers)
    
    if response.status_code == 200:
        access_token = response.text
        return access_token
    else:
        return ''

def request_tts(text):
    endpoint = 'https://eastus.tts.speech.microsoft.com/cognitiveservices/v1'
    access_token = get_token()

    headers = {
        "X-Microsoft-OutputFormat": "riff-24khz-16bit-mono-pcm",
        "Content-Type": "application/ssml+xml",
        "User-Agent": "testForEducation",
        "Authorization": f"Bearer {access_token}"
    }

    # 필터된 텍스트 사용
    data = f"""
    <speak version='1.0' xml:lang='ko-KR'><voice xml:lang='ko-KR' xml:gender='Female' name='ko-KR-SunHiNeural'>
        {text}
    </voice></speak>
    """

    response = requests.post(endpoint, headers=headers, data=data)
    
    if response.status_code == 200:
        file_name = 'response_audio.wav'
        with open(file_name, "wb") as audio_file:
            audio_file.write(response.content)
        return file_name
    else:
        return None

def chatgpt_with_tts(prompt, history):
    response_text, updated_history = chatgpt_response(prompt, history)
    
    # 응답 텍스트 필터링
    filtered_response_text = filter_text(response_text)
    
    # TTS 변환
    audio_file = request_tts(filtered_response_text)
    return audio_file, updated_history

# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    gr.Markdown("<h1 align='center'>ChatGPT with TTS Demo</h1>")
    gr.Dropdown(["원하시는 언어를 선택해 주세요.", "ko-kr", "en-un", 'ja-jp', 'zh-cn', "fr-fr"])
    chatbot = gr.Chatbot()
    with gr.Column():
        with gr.Row():
            user_input = gr.Textbox(show_label=False, placeholder="Type your message here...", scale=3)
            submit_btn = gr.Button("Send", scale=1)
        with gr.Row():
            clear_btn = gr.Button("Clear")
        with gr.Row():
            output_audio = gr.Audio(label="Output Audio", type="filepath", interactive=False)

    # 버튼 클릭 시 이벤트 처리
    submit_btn.click(chatgpt_with_tts, [user_input, chatbot], [output_audio, chatbot])
    clear_btn.click(lambda: None, None, chatbot, queue=False)
    # 엔터 처리
    user_input.submit(chatgpt_with_tts, [user_input, chatbot], [output_audio, chatbot])

# 인터페이스 실행
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7871

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
